# 네이버 자동차 리뷰 분석 및 개선

In [1]:
# from bs4 import BeautifulSoup ## 안돼서 사용안함
import requests
import pandas as pd
import time
import selenium 

print(f'selenium v.{selenium.__version__}')

selenium v.4.38.0


### 정적으로는 리뷰를 가져올 수 없음 -> selenium을 사용해야함

In [2]:
#### ========================================================
#### 이렇게 진행해보니, 숨겨놔서 select로 불러올 수 없음
#### -> Selenium 사용해야함
#### ========================================================
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
# }
# ## 사이트 주소로 객체 만드는 함수
# url = "https://search.naver.com/search.naver?where=nexearch&qvt=0&pkid=128&mra=bjg1&sm=tab_etc&os=40311477&query=2026%20%ED%98%BC%EB%8B%A4%20CR-V%20%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C%20%ED%8F%89%EA%B0%80"  # 평가 페이지 링크

# res = requests.get(url, headers=HEADERS)
# res.raise_for_status()
# soup = BeautifulSoup(res.text, "html.parser")
# soup.select('div.u_cbox_content_wrap ul > li.u_cbox_comment')
# soup

### selenium 시작

In [3]:
## - 모듈 로딩 
from selenium import webdriver # 웹 브라우저(Chrome)를 자동으로 켜서 조작하기 위한 라이브러리
from selenium.webdriver.common.by import By # HTML 요소를 찾을 때 사용 (id, class, css 선택자 등)
from selenium.webdriver.chrome.service import Service # 크롬 드라이버(브라우저 조종 프로그램)를 연결해 주는 도구
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 자동으로 다운로드해 주는 라이브러리

# 웹 페이지가 뜰 때까지 / 특정 요소가 나올 때까지 기다려주는 도구
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# driver = webdriver.Chrome()

# ## 예시 차량 평가 페이지 -> 바로 열기
# driver.get('https://search.naver.com/search.naver?where=nexearch&qvt=0&pkid=128&mra=bjg1&sm=tab_etc&os=40299089&query=2026%20%EB%A9%94%EB%A5%B4%EC%84%B8%EB%8D%B0%EC%8A%A4-%EB%B2%A4%EC%B8%A0%20GLE%20%ED%8F%89%EA%B0%80')

# print(driver.title) 

In [5]:
# =========================================
# 크롬 드라이버 생성 함수
# =========================================
def create_driver():
    """Selenium용 크롬 드라이버를 생성해서 반환한다."""
    options = webdriver.ChromeOptions()
    # 창 안 띄우고 돌리고 싶으면 아래 주석 해제
    # options.add_argument("--headless=new")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

In [6]:
# =========================================
## 리뷰 총 개수 확인하는 함수
# =========================================
def get_total_review_count(driver):
    try:
        elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "span.rating_people > strong")
            )
        )
        total = int(elem.text.strip())
        return total
    except:
        return 0


In [ ]:
# =========================================
## 리뷰 총 개수랑 똑같아 질 때까지 스크롤 하는 함수
# =========================================
def load_all_reviews_until_full(driver, car_name=""):
    """
    총 리뷰 개수(total_count)를 가져온 후,
    실제 로딩된 리뷰 블록 개수(loaded_count)가 total_count에 도달할 때까지 스크롤.
    """

    # 스크롤 박스 찾기
    scroll_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div._scroll_container")
        )
    )

    # 총 리뷰 개수 가져오기
    total_count = get_total_review_count(driver)
    print(f"[{car_name}] 총 리뷰 수: {total_count}")

    # 3) 로딩된 댓글 개수 확인하며 스크롤 반복
    last_loaded = 0
    same_round = 0

    for _ in range(50):  # 최대 50번 스크롤 시도
        # 로딩된 리뷰 수
        review_blocks = scroll_box.find_elements(
            By.CSS_SELECTOR, "div.u_cbox_comment_box"
        )
        loaded_count = len(review_blocks)

        print(f"현재 로딩된 리뷰 수: {loaded_count} / {total_count}")

        # 이미 다 로딩함 → 중지
        if loaded_count >= total_count:
            print("모든 리뷰 로딩 완료!")
            break

        # 변화가 없음 → 잠깐 기다리고 다시 시도
        if loaded_count == last_loaded:
            same_round += 1
        else:
            same_round = 0

        if same_round >= 3:
            print("3번 연속 리뷰 증가 없음 → 중지(로딩 끝난 것으로 판단)")
            break

        # 스크롤 실행
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight;",
            scroll_box
        )

        last_loaded = loaded_count
        time.sleep(1.1)

    # 로딩 완료 후 리뷰 element 리스트 반환
    return scroll_box.find_elements(By.CSS_SELECTOR, "div.u_cbox_comment_box")


In [8]:
# =========================================
## 리뷰에서 차이름, 평가점수, 리뷰 내용 도출해서 append하는 함수
# =========================================
def scroll_and_collect_reviews(driver, car_name=""):
    review_blocks = load_all_reviews_until_full(driver, car_name)

    data = []
    for rb in review_blocks:
        try:
            score = rb.find_element(By.CSS_SELECTOR, "span.u_cbox_multirating_totalcount_value").text.strip()
        except:
            score = None

        try:
            content = rb.find_element(By.CSS_SELECTOR, "div.u_cbox_text_wrap").text.strip()
        except:
            content = ""

        data.append({
            "car": car_name,
            "score": score,
            "content": content
        })

    print(f"[{car_name}] 최종 저장 리뷰 수: {len(data)}")
    return data


In [9]:
# =========================================
# 특정 차량 페이지에서 '평가' 탭 들어가고 리뷰 수집
# =========================================
def go_to_eval_and_collect(driver):
    """
    전제: driver는 현재 '특정 차량 페이지'에 와 있는 상황
    동작:
      1) '평가'라는 텍스트를 가진 탭/버튼을 찾는다.
      2) 있으면 클릭해서 평가 페이지로 이동.
      3) 평가 페이지에서 scroll_and_collect_reviews() 호출.
    """
    data = []

    # 차량 이름(예: 2026 메르세데스-벤츠 GLE) 가져오기
    try:
        car_name_tag = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "h2.title, strong.title_text") 
            )
        )
        car_name = car_name_tag.text.strip()
    except:
        car_name = ""

    # '평가' 탭/버튼 찾기
    try:
        eval_tab = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.XPATH, "//a[contains(., '평가')]")
            )
        )
    except:
        print(f"[{car_name}] 평가 탭 없음 → 스킵")
        return data  # 빈 리스트 반환

    # '평가' 탭 클릭
    try:
        eval_tab.click()
        time.sleep(2)  # 탭 전환 로딩 대기
    except Exception as e:
        print(f"[{car_name}] 평가 탭 클릭 실패:", e)
        return data

    # 현재 페이지에서 리뷰 스크롤 + 수집
    try:
        data = scroll_and_collect_reviews(driver, car_name=car_name)
    except Exception as e:
        print(f"[{car_name}] 리뷰 수집 중 오류:", e)

    return data


# -------------------------------------

In [10]:
# =========================================
## 차량 이름 출력 함수
# =========================================
CAR_SELECTOR = "div.list_info._panel div.info_area > strong > a"

def collect_car_links_on_page(driver):
    """현재 페이지에서 (차이름, href) 리스트를 뽑음"""
    elems = driver.find_elements(By.CSS_SELECTOR, CAR_SELECTOR)
    results = []
    for e in elems:
        name = e.text.strip()
        href = e.get_attribute("href")
        if href:
            results.append((name, href))
    return results


In [11]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# =========================================
## 오른쪽 화살표 클릭 함수
# =========================================

NEXT_BTN_SELECTOR = "div.cm_paging_area.no_margin a.pg_next"

def go_next_page(driver):
    """오른쪽 화살표(다음 페이지) 버튼을 눌러 다음 페이지로 이동. 성공하면 True, 더 이상 없으면 False."""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, NEXT_BTN_SELECTOR)

        # class 에 'disabled' 같은 게 붙는지 체크 (붙으면 더 이상 못 감)
        cls = next_btn.get_attribute("class") or ""
        if "disabled" in cls:
            return False

        # 현재 첫 번째 자동차 이름을 저장해둠 (페이지가 바뀌었는지 확인용)
        try:
            first_name = driver.find_element(By.CSS_SELECTOR, CAR_SELECTOR).text.strip()
        except:
            first_name = None

        # JS로 클릭 실행 (내부 자바스크립트 함수 호출됨)
        driver.execute_script("arguments[0].click();", next_btn)

        # 페이지 내용이 바뀔 때까지 잠깐 대기
        if first_name:
            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element(
                    (By.CSS_SELECTOR, CAR_SELECTOR),
                    ""
                )
            )
        else:
            # 첫 요소가 없었다면 그냥 잠깐 sleep
            time.sleep(1.5)

        return True

    except NoSuchElementException:
        # next 버튼 자체가 없으면 끝
        return False
    except TimeoutException:
        # 내용이 안 바뀌면 더 진행하지 않음
        return False


In [12]:
# =========================================
## 페이지 넘기면서 페이지 들어가고 평가 들어가는 함수
# =========================================

def crawl_all_cars(start_url, max_pages=10):
    """
    start_url: '자동차' 검색 결과 페이지 URL
    max_pages: 최대 몇 페이지까지 넘길지 (테스트용)
    """

    driver = create_driver()
    driver.get(start_url)
    # time.sleep(3)

    all_links = []   # (차이름, href) 모으는 곳
    seen = set()     # 중복 제거용

    current_page = 1
    while current_page <= max_pages:
        print(f"\n=== {current_page} 페이지 수집 중 ===")

        # 현재 페이지에서 자동차 링크 수집
        page_links = collect_car_links_on_page(driver)
        print("이 페이지에서 찾은 차 개수:", len(page_links))

        for name, href in page_links:
            if href not in seen:
                seen.add(href)
                all_links.append((name, href))

        # 다음 페이지로 이동 시도
        moved = go_next_page(driver)
        if not moved:
            print("더 이상 다음 페이지가 없음 → 종료")
            break

        current_page += 1
        # time.sleep(2)

    print("\n총 수집된 차량 링크 수:", len(all_links))

    # 여기서 all_links 를 가지고 각 차량 상세 페이지 → 평가 탭 → 리뷰 수집
    all_reviews = []

    for idx, (car_name, href) in enumerate(all_links, start=1):
        print(f"\n===== [{idx}/{len(all_links)}] {car_name} ({href}) =====")
        try:
            driver.get(href)
            # time.sleep(2)
            car_reviews = go_to_eval_and_collect(driver)
            all_reviews.extend(car_reviews)
        except Exception as e:
            print("[ERROR] 차량 상세 페이지 처리 중 에러:", e)
            continue

    driver.quit()
    return all_reviews


In [13]:
# =========================================
# 메인 실행부
# =========================================
if __name__ == "__main__":
    START_URL = (
        'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%9E%90%EB%8F%99%EC%B0%A8&ackey=j1rh28hp'
    )

    # '자동차' 검색 페이지에서 최대 10개 모델만 테스트
    reviews = crawl_all_cars(START_URL)

    # pandas DataFrame으로 변환 후 CSV 저장
    df = pd.DataFrame(reviews)
    df.to_csv("naver_car_reviews_all_ffinal.csv", index=False, encoding="utf-8-sig")

    print("\n전체 저장 완료! 총 리뷰 수:", len(df))


=== 1 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 16

=== 2 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 24

=== 3 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 32

=== 4 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 40

=== 5 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 48

=== 6 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 56

=== 7 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 64

=== 8 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 72

=== 9 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 80

=== 10 페이지 수집 중 ===
이 페이지에서 찾은 차 개수: 88

총 수집된 차량 링크 수: 88

===== [1/88] 2026 혼다 CR-V 하이브리드 (https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os=40311477&qvt=0&query=2026%20%ED%98%BC%EB%8B%A4%20CR-V%20%ED%95%98%EC%9D%B4%EB%B8%8C%EB%A6%AC%EB%93%9C) =====
[2026 혼다 CR-V 하이브리드] 총 리뷰 수: 61
현재 로딩된 리뷰 수: 10 / 61
현재 로딩된 리뷰 수: 20 / 61
현재 로딩된 리뷰 수: 30 / 61
현재 로딩된 리뷰 수: 40 / 61
현재 로딩된 리뷰 수: 50 / 61
현재 로딩된 리뷰 수: 60 / 61
현재 로딩된 리뷰 수: 61 / 61
모든 리뷰 로딩 완료!
[2026 혼다 CR-V 하이브리드] 최종 저장 리뷰 수: 61

===== [2/88] 2026 토요타 프리우스 플러그... (https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjg1&pkid=128&os=402

# -----------------------------------------------
